<a href="https://colab.research.google.com/github/ekity1002/DL4US_FINAL_WORK/blob/master/%E6%A5%BD%E5%A4%A9%E3%83%AC%E3%82%B7%E3%83%94%E5%8F%96%E5%BE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# -*- coding:utf-8 -*-

import requests
import csv
from tqdm import tqdm
from bs4 import BeautifulSoup
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline


# 訓練データ

In [0]:
# 一覧取得用
N = 1
ZAIRYOU = []
INSTRUCTIONS = []
IMG_PATHS = []

def write_tsv(path, rows):
    with open(path, 'a') as f:
        writer = csv.writer(f, delimiter='\t')
        for row in rows:
            writer.writerow(row)

def scrape(start_url, page_range):
    global N
    for i in tqdm(page_range):
        res = requests.get(f"{start_url}"+f"/{i}/")
        soup = BeautifulSoup(res.text, 'html.parser')

        # メニュー名のリンクリストを作成
        menu_links = [n for n in soup.select('.clearfix #recipe_link, #recipe_detail_link')]
        urls = [l['href'] for l in menu_links]

        # メニュー内容のスクレイピング
        for url in urls:
            # スクレイピング対象の URL にリクエストを送り HTML を取得する
            #print(url)
            res = requests.get('https://recipe.rakuten.co.jp/'+url)

            # レスポンスの HTML から BeautifulSoup オブジェクトを作る
            soup = BeautifulSoup(res.text, 'html.parser')

            # CSS セレクターを使って author を全て取得する
            # おつまみかどうか判定
            # author_names = [n.get_text(strip=True) for n in soup.select('a[sctag="recipe_detail_subcategory_medium"]')]
            # print(author_names)
            # not おつまみ continue

            # 画像
            imgs = [n for n in soup.select('img[itemprop="image"]')]
            if len(imgs)==0 :
                continue
            src = imgs[0]['src']
            r = requests.get(src)# title タグの文字列を取得する
            img_path = f'img{N}.jpg'
            with open(img_path, 'wb') as f:
                f.write(r.content)
            IMG_PATHS.append([img_path])
            
            # タイトル
            title_text = soup.find('h1').get_text(strip=True)
            #print(title_text.split()[0])

            # 材料
            zairyou = [n.get_text(strip=True) for n in soup.select('li[itemprop="ingredients"] a.name, div.name')]
            #print(zairyou)
            # カテゴリ(ビール/日本酒/ワイン/など)
            # category = [n.get_text(strip=True) for n in soup.select('a[sctag="recipe_detail_subcategory_small"]')]
            ZAIRYOU.append(zairyou)
            #print(ZAIRYOU)
            

            # 手順
            instructions = [n.get_text(strip=True) for n in soup.select('p.stepMemo')]
            #print(instructions)
            INSTRUCTIONS.append(instructions)

            N += 1
            #print("@@@@@@@@@@@@@@@@@@@@@@@@@")

scrape('https://recipe.rakuten.co.jp/category/26-260-1068', range(1,51)) #ビールに合う
scrape('https://recipe.rakuten.co.jp/category/26-260-1069', range(1,51)) #ワイン
scrape('https://recipe.rakuten.co.jp/category/26-260-1070', range(1,51)) #日本酒
scrape('https://recipe.rakuten.co.jp/category/26-260-1071', range(1,51)) #焼酎
print(len(ZAIRYOU))
print(len(INSTRUCTIONS))
print(len(IMG_PATHS))



 24%|██▍       | 12/50 [04:07<14:04, 22.23s/it]

 26%|██▌       | 13/50 [04:34<14:37, 23.72s/it]

 28%|██▊       | 14/50 [05:01<14:47, 24.65s/it]

 30%|███       | 15/50 [05:22<13:43, 23.54s/it]

 32%|███▏      | 16/50 [05:46<13:26, 23.72s/it]

 34%|███▍      | 17/50 [06:13<13:39, 24.82s/it]

 36%|███▌      | 18/50 [06:39<13:27, 25.24s/it]

 38%|███▊      | 19/50 [07:01<12:31, 24.25s/it]

 40%|████      | 20/50 [07:27<12:24, 24.81s/it]

 42%|████▏     | 21/50 [07:56<12:33, 25.97s/it]

 44%|████▍     | 22/50 [08:23<12:13, 26.19s/it]

 46%|████▌     | 23/50 [08:47<11:34, 25.71s/it]

In [0]:
# 保存
INST_PATH = '/content/drive/My Drive/Colab Notebooks/data/INSTRUCTIONS.tsv'
ZAIRYOU_PATH = '/content/drive/My Drive/Colab Notebooks/data/ZAIRYOU.tsv'
IMGTSV_PATH = '/content/drive/My Drive/Colab Notebooks/data/IMG_PATH.tsv'
write_tsv(INST_PATH, INSTRUCTIONS)
write_tsv(ZAIRYOU_PATH, ZAIRYOU)
write_tsv(IMGTSV_PATH, IMG_PATHS)

In [0]:
!zip train_img.zip *.jpg
!cp train_img.zip '/content/drive/My Drive/Colab Notebooks/data/'

#テストデータ

In [0]:
# 一覧取得用
N = 1
ZAIRYOU = []
INSTRUCTIONS = []
IMG_PATHS = []
scrape('https://recipe.rakuten.co.jp/category/26-260-1068', range(52,61)) #ビールに合う
scrape('https://recipe.rakuten.co.jp/category/26-260-1069', range(52,61)) #ビールに合う
scrape('https://recipe.rakuten.co.jp/category/26-260-1070', range(52,61)) #ビールに合う
scrape('https://recipe.rakuten.co.jp/category/26-260-1071', range(52,61)) #ビールに合う
print(len(ZAIRYOU))
print(len(INSTRUCTIONS))
print(len(IMG_PATHS))

In [0]:
# ファイル保存
INST_PATH = '/content/drive/My Drive/Colab Notebooks/data/test_INSTRUCTIONS.tsv'
ZAIRYOU_PATH = '/content/drive/My Drive/Colab Notebooks/data/test_ZAIRYOU.tsv'
IMGTSV_PATH = '/content/drive/My Drive/Colab Notebooks/data/test_IMG_PATH.tsv'
write_tsv(INST_PATH, INSTRUCTIONS)
write_tsv(ZAIRYOU_PATH, ZAIRYOU)
write_tsv(IMGTSV_PATH, IMG_PATHS)

In [0]:
!zip test_img.zip test*.jpg
!cp test_img.zip '/content/drive/My Drive/Colab Notebooks/data/'